# 🏎 F1 - PREDICTING GP Winners 🏎 NEURAL NETWORK MODEL

In [18]:
# importing libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.layers import Dense


In [2]:
# Reading in the preprocessed ml dataframe from GCS bucket
file_path = "https://storage.googleapis.com/f1_winners_historical/q_ml.csv"
q_ = pd.read_csv(file_path,
                 sep=",", low_memory=False)

In [3]:
q_

,Unnamed: 0,Race_Winner,Race_AVG_Lap_Time_sec,DriverNumber,Q_LapTime(s),LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,Grand Prix_qatar,Grand Prix_russia,Grand Prix_sakhir,Grand Prix_saudi-arabia,Grand Prix_singapore,Grand Prix_spain,Grand Prix_styria,Grand Prix_turkey,Grand Prix_tuscany,Grand Prix_united-states
0,0,SebastianVettel,92.637931,27,112.720,1.0,NaN,262.434,NaN,42.085,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,SebastianVettel,92.637931,27,112.720,1.0,NaN,262.434,NaN,42.085,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,SebastianVettel,92.637931,27,85.348,2.0,1.0,NaN,NaN,28.492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,SebastianVettel,92.637931,27,85.348,2.0,1.0,NaN,NaN,28.492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,SebastianVettel,92.637931,27,116.609,3.0,1.0,NaN,NaN,40.615,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37318,37318,MaxVerstappen,90.034483,63,NaN,13.0,5.0,3877.412,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37319,37319,MaxVerstappen,90.034483,63,84.152,14.0,5.0,NaN,NaN,17.256,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37320,37320,MaxVerstappen,90.034483,63,101.723,15.0,5.0,NaN,4181.281,20.542,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37321,37321,MaxVerstappen,90.034483,63,NaN,16.0,6.0,4353.278,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Taking a look at the data and setting option to view all columns in df
pd.set_option('display.max_columns', None)

In [5]:
# Had dropped the 'Driver' column previously but wanted to map it to 'Race_Winner' column.
# Creating one column out of the one hot encoded column with drivers

one_hot_columns = ['Driver_ALB','Driver_ALO','Driver_BOT','Driver_DEV','Driver_ERI','Driver_FIT','Driver_GAS','Driver_GIO','Driver_GRO','Driver_HAM','Driver_HAR','Driver_HUL','Driver_KUB','Driver_KVY','Driver_LAT','Driver_LAW','Driver_LEC','Driver_MAG','Driver_MAZ','Driver_MSC','Driver_NOR','Driver_OCO','Driver_PER','Driver_PIA','Driver_RAI','Driver_RIC','Driver_RUS','Driver_SAI','Driver_SAR','Driver_SIR','Driver_STR','Driver_TSU','Driver_VAN','Driver_VER','Driver_VET','Driver_ZHO']

# Function to combine one-hot encoded columns
def combine_one_hot_columns(row):
    for col in one_hot_columns:
        if row[col] == 1:
            return col.split('_')[1]  # Extract driver name from column name
    return 'Unknown'

# Create the new column (keep the original columns)
q_['Driver'] = q_.apply(combine_one_hot_columns, axis=1)

q_

,Unnamed: 0,Race_Winner,Race_AVG_Lap_Time_sec,DriverNumber,Q_LapTime(s),LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,SpeedST,TyreLife,TrackStatus,Year,Event_id,Team_Alfa Romeo,Team_Alfa Romeo Racing,Team_AlphaTauri,Team_Alpine,Team_Aston Martin,Team_Ferrari,Team_Force India,Team_Haas F1 Team,Team_McLaren,Team_Mercedes,Team_Racing Point,Team_Red Bull Racing,Team_Renault,Team_Sauber,Team_Toro Rosso,Team_Williams,Compound_HARD,Compound_HYPERSOFT,Compound_INTERMEDIATE,Compound_MEDIUM,Compound_SOFT,Compound_SUPERSOFT,Compound_ULTRASOFT,Compound_UNKNOWN,Compound_WET,Driver_ALB,Driver_ALO,Driver_BOT,Driver_DEV,Driver_ERI,Driver_FIT,Driver_GAS,Driver_GIO,Driver_GRO,Driver_HAM,Driver_HAR,Driver_HUL,Driver_KUB,Driver_KVY,Driver_LAT,Driver_LAW,Driver_LEC,Driver_MAG,Driver_MAZ,Driver_MSC,Driver_NOR,Driver_OCO,Driver_PER,Driver_PIA,Driver_RAI,Driver_RIC,Driver_RUS,Driver_SAI,Driver_SAR,Driver_SIR,Driver_STR,Driver_TSU,Driver_VAN,Driver_VER,Driver_VET,Driver_ZHO,IsPersonalBest_True,Q_FreshTyre_True,Q_Hours,Q_Minutes,Q_AirTemp,Q_Humidity,Q_Pressure,Q_TrackTemp,Q_WindDirection,Q_WindSpeed,Q_Rainfall_True,Q1,Q2,Q3,#QLaps,Q_Pos_1,Q_Pos_10,Q_Pos_11,Q_Pos_12,Q_Pos_13,Q_Pos_14,Q_Pos_15,Q_Pos_16,Q_Pos_17,Q_Pos_18,Q_Pos_19,Q_Pos_2,Q_Pos_20,Q_Pos_3,Q_Pos_4,Q_Pos_5,Q_Pos_6,Q_Pos_7,Q_Pos_8,Q_Pos_9,Q_Pos_NC,Q_Pos_RT,Grand Prix_70th-anniversary,Grand Prix_abu-dhabi,Grand Prix_australia,Grand Prix_austria,Grand Prix_azerbaijan,Grand Prix_bahrain,Grand Prix_belgium,Grand Prix_brazil,Grand Prix_canada,Grand Prix_china,Grand Prix_eifel,Grand Prix_emilia-romagna,Grand Prix_france,Grand Prix_germany,Grand Prix_great-britain,Grand Prix_hungary,Grand Prix_italy,Grand Prix_japan,Grand Prix_las-vegas,Grand Prix_mexico,Grand Prix_miami,Grand Prix_monaco,Grand Prix_netherlands,Grand Prix_portugal,Grand Prix_qatar,Grand Prix_russia,Grand Prix_sakhir,Grand Prix_saudi-arabia,Grand Prix_singapore,Grand Prix_spain,Grand Prix_styria,Grand Prix_turkey,Grand Prix_tuscany,Grand Prix_united-states,Driver
0,0,SebastianVettel,92.637931,27,112.720,1.0,NaN,262.434,NaN,42.085,28.028,42.607,226.0,262.0,293.0,194.0,NaN,1.0,2018,979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,6,24.6,59.1,1007.9,31.7,0,1.3,0.0,83.782,83.544,83.532,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HUL
1,1,SebastianVettel,92.637931,27,112.720,1.0,NaN,262.434,NaN,42.085,28.028,42.607,226.0,262.0,293.0,194.0,NaN,1.0,2018,979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,6,24.6,59.1,1007.9,31.7,0,1.3,0.0,83.782,83.544,83.532,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HUL
2,2,SebastianVettel,92.637931,27,85.348,2.0,1.0,NaN,NaN,28.492,23.123,33.733,276.0,286.0,296.0,302.0,1.0,1.0,2018,979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,7,24.7,59.5,1007.9,31.8,318,0.9,0.0,83.782,83.544,83.532,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [6]:
# Getting unique values for race winners and quaification drivers
Race_w=q_.Race_Winner.unique()
Race_w

q_drv=q_.Driver.unique()
q_drv

array(['HUL', 'LEC', 'ERI', 'GRO', 'SAI', 'MAG', 'PER', 'HAR', 'OCO',
       'RAI', 'VET', 'HAM', 'BOT', 'STR', 'VAN', 'GAS', 'SIR', 'ALO',
       'RIC', 'VER', 'NOR', 'KUB', 'RUS', 'GIO', 'KVY', 'ALB', 'LAT',
       'FIT', 'MAZ', 'MSC', 'TSU', 'ZHO', 'DEV', 'PIA', 'SAR', 'LAW'],
      dtype=object)

In [7]:
# Create a dictionary for individual mappings for Driver column
driver_mapping = {
    'SebastianVettel': 'VET',
    'DanielRicciardo': 'RIC',
    'LewisHamilton': 'HAM',
    'MaxVerstappen': 'VER',
    'KimiRäikkönen': 'RAI',
    'ValtteriBottas': 'BOT',
    'CharlesLeclerc': 'LEC',
    'PierreGasly': 'GAS',
    'SergioPerez': 'PER',
    'EstebanOcon': 'OCO',
    'CarlosSainz': 'SAI',
    'GeorgeRussell': 'RUS'
}

# Function to apply the mapping to each driver name in the DataFrame
def map_driver(driver_name):
    return driver_mapping.get(driver_name, driver_name)

# Apply the mapping to the 'Driver' column
q_['Race_Winner'] = q_['Race_Winner'].apply(map_driver)

In [8]:
# Dropping empty column
q_.drop(columns=['Unnamed: 0'], inplace=True)


In [9]:
df=q_

In [10]:



# Check and fill NaN values
df.fillna(0, inplace=True)

# Check for and replace infinite values
df.replace([np.inf, -np.inf], 0, inplace=True)

# Encode categorical columns to numerical values
le = LabelEncoder()
df['Driver'] = le.fit_transform(df['Driver'])
df['Race_Winner'] = le.fit_transform(df['Race_Winner'])

# Create features and target
X = df[['Event_id', 'Driver']]
y = df['Race_Winner']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building the neural network model
model = Sequential([
    Dense(96, input_shape=(X_train_scaled.shape[1],), activation='relu'),
    Dense(96, activation='relu'),
    Dense(96, activation='relu'),
    Dense(66, activation='relu'),
    Dense(66, activation='relu'),
    Dense(66, activation='relu'),
    Dense(33, activation='relu'),
    Dense(33, activation='relu'),
    Dense(33, activation='relu'),
    Dense(len(y.unique()), activation='softmax')  # Output layer: one neuron per class
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=63, batch_size=3, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}')

# Make predictions for each 'Event_id'
event_ids = df['Event_id'].unique()
winner_predictions = []
for event_id in event_ids:
    event_data = df[df['Event_id'] == event_id]
    event_X = scaler.transform(event_data[['Event_id', 'Driver']])
    event_predictions = model.predict(event_X)
    predicted_class = np.argmax(event_predictions, axis=1)
    winner = le.inverse_transform([predicted_class.max()])
    winner_predictions.append({'Event_id': event_id, 'Race_winner_prediction': winner[0]})

# Create the final dataframe
winner_predictions_df = pd.DataFrame(winner_predictions)

# Save the results
#winner_predictions_df.to_csv('winner_predictions.csv', index=False)

Epoch 1/63
7962/7962 [==============================] - 27s 3ms/step - loss: 1.2440 - accuracy: 0.6052 - val_loss: 1.1350 - val_accuracy: 0.5970
Epoch 2/63
7962/7962 [==============================] - 28s 4ms/step - loss: 1.0519 - accuracy: 0.6320 - val_loss: 0.9941 - val_accuracy: 0.6363
Epoch 3/63
7962/7962 [==============================] - 28s 4ms/step - loss: 0.9923 - accuracy: 0.6388 - val_loss: 1.0008 - val_accuracy: 0.6330
Epoch 4/63
7962/7962 [==============================] - 25s 3ms/step - loss: 0.9344 - accuracy: 0.6430 - val_loss: 0.8822 - val_accuracy: 0.6432
Epoch 5/63
7962/7962 [==============================] - 24s 3ms/step - loss: 0.8746 - accuracy: 0.6548 - val_loss: 0.8056 - val_accuracy: 0.6681
Epoch 6/63
7962/7962 [==============================] - 27s 3ms/step - loss: 0.8362 - accuracy: 0.6561 - val_loss: 0.7845 - val_accuracy: 0.6661
Epoch 7/63
7962/7962 [==============================] - 22s 3ms/step - loss: 0.7961 - accuracy: 0.6683 - val_loss: 0.9469 - val_ac

In [11]:
df

,Race_Winner,Race_AVG_Lap_Time_sec,DriverNumber,Q_LapTime(s),LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,SpeedFL,SpeedST,TyreLife,TrackStatus,Year,Event_id,Team_Alfa Romeo,Team_Alfa Romeo Racing,Team_AlphaTauri,Team_Alpine,Team_Aston Martin,Team_Ferrari,Team_Force India,Team_Haas F1 Team,Team_McLaren,Team_Mercedes,Team_Racing Point,Team_Red Bull Racing,Team_Renault,Team_Sauber,Team_Toro Rosso,Team_Williams,Compound_HARD,Compound_HYPERSOFT,Compound_INTERMEDIATE,Compound_MEDIUM,Compound_SOFT,Compound_SUPERSOFT,Compound_ULTRASOFT,Compound_UNKNOWN,Compound_WET,Driver_ALB,Driver_ALO,Driver_BOT,Driver_DEV,Driver_ERI,Driver_FIT,Driver_GAS,Driver_GIO,Driver_GRO,Driver_HAM,Driver_HAR,Driver_HUL,Driver_KUB,Driver_KVY,Driver_LAT,Driver_LAW,Driver_LEC,Driver_MAG,Driver_MAZ,Driver_MSC,Driver_NOR,Driver_OCO,Driver_PER,Driver_PIA,Driver_RAI,Driver_RIC,Driver_RUS,Driver_SAI,Driver_SAR,Driver_SIR,Driver_STR,Driver_TSU,Driver_VAN,Driver_VER,Driver_VET,Driver_ZHO,IsPersonalBest_True,Q_FreshTyre_True,Q_Hours,Q_Minutes,Q_AirTemp,Q_Humidity,Q_Pressure,Q_TrackTemp,Q_WindDirection,Q_WindSpeed,Q_Rainfall_True,Q1,Q2,Q3,#QLaps,Q_Pos_1,Q_Pos_10,Q_Pos_11,Q_Pos_12,Q_Pos_13,Q_Pos_14,Q_Pos_15,Q_Pos_16,Q_Pos_17,Q_Pos_18,Q_Pos_19,Q_Pos_2,Q_Pos_20,Q_Pos_3,Q_Pos_4,Q_Pos_5,Q_Pos_6,Q_Pos_7,Q_Pos_8,Q_Pos_9,Q_Pos_NC,Q_Pos_RT,Grand Prix_70th-anniversary,Grand Prix_abu-dhabi,Grand Prix_australia,Grand Prix_austria,Grand Prix_azerbaijan,Grand Prix_bahrain,Grand Prix_belgium,Grand Prix_brazil,Grand Prix_canada,Grand Prix_china,Grand Prix_eifel,Grand Prix_emilia-romagna,Grand Prix_france,Grand Prix_germany,Grand Prix_great-britain,Grand Prix_hungary,Grand Prix_italy,Grand Prix_japan,Grand Prix_las-vegas,Grand Prix_mexico,Grand Prix_miami,Grand Prix_monaco,Grand Prix_netherlands,Grand Prix_portugal,Grand Prix_qatar,Grand Prix_russia,Grand Prix_sakhir,Grand Prix_saudi-arabia,Grand Prix_singapore,Grand Prix_spain,Grand Prix_styria,Grand Prix_turkey,Grand Prix_tuscany,Grand Prix_united-states,Driver
0,11,92.637931,27,112.720,1.0,0.0,262.434,0.000,42.085,28.028,42.607,226.0,262.0,293.0,194.0,0.0,1.0,2018,979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,6,24.6,59.1,1007.9,31.7,0,1.3,0.0,83.782,83.544,83.532,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
1,11,92.637931,27,112.720,1.0,0.0,262.434,0.000,42.085,28.028,42.607,226.0,262.0,293.0,194.0,0.0,1.0,2018,979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,6,24.6,59.1,1007.9,31.7,0,1.3,0.0,83.782,83.544,83.532,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
2,11,92.637931,27,85.348,2.0,1.0,0.000,0.000,28.492,23.123,33.733,276.0,286.0,296.0,302.0,1.0,1.0,2018,979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,7,24.7,59.5,1007.9,31.8,318,0.9,0.0,83.782,83.544,83.532,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
3,11,92.637931,27,85.34

In [12]:
pd.set_option('display.max_rows', None)
winner_predictions_df

,Event_id,Race_winner_prediction
0,979,VET
1,980,VET
2,981,RIC
3,982,HAM
4,983,HAM
5,984,RIC
6,985,HAM
7,986,HAM
8,987,VER
9,988,VET


In [16]:
# Saving as CSV in local file system
winner_predictions_df.to_csv('winner_predictions_df.csv')

In [21]:
# Function to upload or overwrite csv file to desired GCP Cloud Storage Bucket
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='gcp-creds.json'


def upload_file_to_bucket(bucket_name, source_file_name, destination_blob_name):

    """Uploads a file to the GCP bucket."""

    client = storage.Client()
    bucket = client.get_bucket('f1_winners_historical')
    bucket_name = "f1_winners_historical"
    source_file_name = "winner_predictions_df.csv"
    destination_blob_name = "winner_predictions_df.csv"

    client = storage.Client()
    bucket = client.bucket('f1_winners_historical')
    blob = bucket.blob('winner_predictions_df.csv')

    blob.upload_from_filename('winner_predictions_df.csv')

    print(
        "File {} uploaded to {}.".format(
            source_file_name, bucket_name))

In [13]:
# Reading in the preprocessed dataframe from GCS bucket
file_path = "https://storage.googleapis.com/f1_winners_historical/q_ml.csv"
df1 = pd.read_csv(file_path,
                 sep=",", low_memory=False)

In [14]:
df1=df1[['Race_Winner','Event_id']]
df1=df1.drop_duplicates()
df1


,Race_Winner,Event_id
0,SebastianVettel,979
536,SebastianVettel,980
767,DanielRicciardo,981
1005,LewisHamilton,982
1308,LewisHamilton,983
1598,DanielRicciardo,984
1976,SebastianVettel,985
2251,LewisHamilton,986
2564,MaxVerstappen,987
2882,SebastianVettel,988


In [15]:
pred_df=pd.merge(winner_predictions_df, df1, left_on=['Event_id'], right_on = ['Event_id'])
pred_df

,Event_id,Race_winner_prediction,Race_Winner
0,979,VET,SebastianVettel
1,980,VET,SebastianVettel
2,981,RIC,DanielRicciardo
3,982,HAM,LewisHamilton
4,983,HAM,LewisHamilton
5,984,RIC,DanielRicciardo
6,985,HAM,SebastianVettel
7,986,HAM,LewisHamilton
8,987,VER,MaxVerstappen
9,988,VET,SebastianVettel
